# Working with real-world datasets and built-in SQL functions

### Connect to Db2 database on Cloud using Python. Import the ibm_db Python library.

In [1]:
import ibm_db

### Identify the database connection credentials

Connecting to dashDB or DB2 database requires the following information:

* Driver Name
* Database name
* Host DNS name or IP address
* Host port
* Connection protocol
* User ID (or username)
* User Password

In [3]:
dsn_hostname = "0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud"  # e.g.: "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = "yhm90744"        # e.g. "abc12345"
dsn_pwd = "jl9J6lYUb2AFH4U6"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "31198"               # e.g. "32733" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"              #i.e. "SSL"

### Create the DB2 database connection

Ibm_db API uses the IBM Data Server Driver for ODBC and CLI APIs to connect to IBM DB2 and Informix.

Lets build the dsn connection string using the credentials you entered above

In [4]:
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

#print the connection string to check correct values are specified
print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud;PORT=31198;PROTOCOL=TCPIP;UID=yhm90744;PWD=jl9J6lYUb2AFH4U6;SECURITY=SSL;


**Now establish the connection to the database**

In [5]:
#Create database connection

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  yhm90744 on host:  0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud


In [6]:
#Retrieve Metadata for the Database Server
server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

DBMS_NAME:  DB2/LINUXX8664
DBMS_VER:   11.05.0500
DB_NAME:    BLUDB


In [7]:
#Retrieve Metadata for the Database Client / Driver
client = ibm_db.client_info(conn)

print ("DRIVER_NAME:          ", client.DRIVER_NAME) 
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)

DRIVER_NAME:           libdb2.a
DRIVER_VER:            11.05.0600
DATA_SOURCE_NAME:      BLUDB
DRIVER_ODBC_VER:       03.51
ODBC_VER:              03.01.0000
ODBC_SQL_CONFORMANCE:  EXTENDED
APPL_CODEPAGE:         1208
CONN_CODEPAGE:         1208


## Task 4: Create a table in the database

In [54]:
#Lets first drop the table INSTRUCTOR in case it exists from a previous attempt

dropstat = ibm_db.exec_immediate(conn, "DROP TABLE Instructor")

In [55]:
createTable = ibm_db.exec_immediate(conn , "CREATE TABLE Instructor( Id integer PRIMARY KEY NOT NULL , FName varchar(20) , LName varchar(20) , City varchar(20) , CCode char(2))")

createTable

## Task 5: Insert data into the table

In [56]:
insertTable1 = ibm_db.exec_immediate(conn , "INSERT INTO Instructor VALUES (1, 'Rav' , 'Ahuja' , 'Toronto', 'CA')")

In [57]:
insertTable2 = ibm_db.exec_immediate(conn , "INSERT INTO Instructor VALUES (2, 'Raul' , 'Chong' , 'Markham', 'CA') , (3, 'Hima' , 'Vasudevan' , 'Chicago', 'US')")

## Task 6: Query data in the table

In [58]:
selectTable = ibm_db.exec_immediate( conn , "SELECT * FROM Instructor")

ibm_db.fetch_both(selectTable)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'Toronto',
 3: 'Toronto',
 'CCODE': 'CA',
 4: 'CA'}

**Fetch the rest of the rows and print the ID and FNAME for those rows**

In [59]:
while ibm_db.fetch_row(selectTable) != False:
    print("ID:" , ibm_db.result(selectTable , 0) , "::  FName:" , ibm_db.result(selectTable , 1))

ID: 2 ::  FName: Raul
ID: 3 ::  FName: Hima


**now write and execute an update statement that changes the Rav's CITY to MOOSETOWN**

In [61]:
updateTable = ibm_db.exec_immediate(conn , "UPDATE Instructor SET City = 'Moosetown' WHERE ID=1 ")

### Task 7: Retrieve data into Pandas

In [63]:
import pandas
import ibm_db_dbi

In [64]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [68]:

#retrieve the query results into a pandas dataframe
df = pandas.read_sql("select * from INSTRUCTOR", pconn)

#print just the LNAME for first row in the pandas data frame
df

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,Moosetown,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


In [69]:
df.FNAME

0     Rav
1    Raul
2    Hima
Name: FNAME, dtype: object

In [70]:
# you can use the shape method to see how many rows and columns are in the dataframe

df.shape

(3, 5)

### Close the Connection

We free all resources by closing the connection. Remember that it is always important to close connections so that we can avoid unused connections taking up resources.

In [21]:
ibm_db.close(conn)

True